In [1]:
import pickle, random

### **Load Data**

In [2]:
with open('/home/maryam/llamaPersonaResp/Data/common/train_data_id.pkl', 'rb') as f:
  train_data_id = pickle.load(f)
with open('/home/maryam/llamaPersonaResp/Data/common/test_data_id.pkl', 'rb') as f:
  test_data_id = pickle.load(f)
with open('/home/maryam/llamaPersonaResp/Data/common/dev_data_id.pkl', 'rb') as f:
  dev_data_id = pickle.load(f)

In [3]:
with open('/home/maryam/llamaPersonaResp/Data/common/train_response_user.pkl', 'rb') as f:
    train_response_user = pickle.load(f)
with open('/home/maryam/llamaPersonaResp/Data/common/test_response_user.pkl', 'rb') as f:
    test_response_user = pickle.load(f)
with open('/home/maryam/llamaPersonaResp/Data/common/dev_response_user.pkl', 'rb') as f:
    dev_response_user = pickle.load(f)

In [4]:
with open('/home/maryam/llamaPersonaResp/Data/common/user_messages.pkl', 'rb') as f:
    user_messages = pickle.load(f)
with open('/home/maryam/llamaPersonaResp/Data/common/wikipedia_conv.pkl', 'rb') as f:
    wikipedia = pickle.load(f)

### **Functions to Generate Train/Test/Dev Data**

In [5]:
# ========================================================
# This function maps the ids in the DH to their real texts
# ========================================================
def id_to_text(id_histories, dataset):
    utterance_dict = {}
    for _ , conversation in dataset.items():
        for utterance in conversation:
            utterance_dict[utterance['id']] = utterance['text']

    text_histories = []
    for history, response_id in id_histories:
        text_history = [utterance_dict[utt_id] for utt_id in history if utt_id in utterance_dict]
        true_response = utterance_dict.get(response_id, "")
        text_history = '__eou__'.join(text_history)
        text_histories.append((text_history, true_response))


    return text_histories, utterance_dict

In [6]:
def generate_final_data(text_histories, wrong_response):
  final_list = []
  for (history, correct_response), wrong_response in zip(text_histories, wrong_response):
      # Randomly choose between the correct and wrong response
      if random.choice([True, False]):
          chosen_response = correct_response
          label = 1  # 1 if true response chosen
      else:
          chosen_response = wrong_response
          label = 0  # 0 if wrong response chosen
      # Append a tuple to the final list: (history, chosen_response, label)
      final_list.append((history, chosen_response, label))

  return final_list

In [7]:
# =============================================================
# Creating Distractor list among same user's all responses.
# =============================================================
def get_distractor(response_user):
  distractor_list = []
  for user in response_user:
    distractor = random.choice(list(user_messages[user]))
    distractor_list.append(distractor)

  return distractor_list

In [8]:
def generate_final_data(text_histories, wrong_response):
  final_list = []
  for (history, correct_response), wrong_response in zip(text_histories, wrong_response):
      # Randomly choose between the correct and wrong response
      if random.choice([True, False]):
          chosen_response = correct_response
          label = 1  # 1 if true response chosen
      else:
          chosen_response = wrong_response
          label = 0  # 0 if wrong response chosen
      # Append a tuple to the final list: (history, chosen_response, label)
      final_list.append((history, chosen_response, label))

  return final_list

In [9]:
train_distractor = get_distractor(train_response_user)
dev_distractor = get_distractor(dev_response_user)
test_distractor = get_distractor(test_response_user)

In [10]:
train_distractor = get_distractor(train_response_user)
dev_distractor = get_distractor(dev_response_user)
test_distractor = get_distractor(test_response_user)

In [11]:
train_text_histories, utterance_dict = id_to_text(train_data_id, wikipedia)
dev_text_histories, utterance_dict = id_to_text(dev_data_id, wikipedia)
test_text_histories, utterance_dict = id_to_text(test_data_id, wikipedia)

In [12]:
final_train_data = generate_final_data(train_text_histories, train_distractor)
final_dev_data = generate_final_data(dev_text_histories, dev_distractor)

In [13]:
final_test_data = [
    [history[0], history[1], wrong_response]
    for history, wrong_response in zip(test_text_histories, test_distractor)
]

### **Save the data w.o userprofiles**

In [15]:
with open('/home/maryam/llamaPersonaResp/Data/data_w.o_profile/final_train.pkl', 'wb') as f:
  pickle.dump(final_train_data, f)

with open('/home/maryam/llamaPersonaResp/Data/data_w.o_profile/final_dev.pkl', 'wb') as f:
  pickle.dump(final_dev_data, f)

with open('/home/maryam/llamaPersonaResp/Data/data_w.o_profile/final_test.pkl', 'wb') as f:
  pickle.dump(final_test_data, f)

In [16]:
final_train_data[0]

('I wanted to let you know that I\'ve archived the FAC for [[Merry Xmas Everybody]].  This is primarily to give you time to track down a copy of the Holder bio and possibly the Pedler book.  If you get the books more quickly, feel free to ignore the \\"several weeks before renomination\\" rule and bring the article book sooner. Good luck! __eou__Kinda pointless as the article was intended to be nominated for a main page appearance on Christmas Day. I don\'t need a copy of the Pedler book, it\'s on Google books, and the Holder bio is unlikely to add anything to it. The best plan is to just leave it, and know that it is a FA just without the pretty star. \'\'\'__eou__Without \\"the pretty star\\" it isn\'t an FA Majorly. ;-) Why not take Karanacs up on her offer? I would. --__eou__There is no way I\'ll be able to get either book in enough time. \'\'\'__eou__Christmas comes every year.... Perhaps it can be featured next year? ',
 "Yeah sure, just request at [[WP:RFA]]. Good luck <s>Sam<\\